Note: This is only a **temporary** notebook that is used for development purposes. 

In [1]:
%load_ext autoreload
%autoreload 2


import os
import sys
import numpy as np
import pandas as pd

from numpy.matlib import repmat

from config import config
import fit_ac_fc as f

In [ ]:
ab = f.load_abundance_thresholds(os.path.join(config.A549_test, \
        "A549_abundance_thresholds.txt"))
tps = f.load_timepoint_counts(os.path.join(config.A549_test, \
        "A549_timepoint_counts.txt"))

In [ ]:
_tps, names = f.convert_timepoint_counts(tps)
_abundance = f.convert_abundance_thresholds(ab)

In [ ]:
# For testing only

_tps = _tps.iloc[:100,:]
_abundance = _abundance.iloc[:100,:]

In [ ]:
tp_tmp = np.array([_tps[1].as_matrix(), _tps[2].as_matrix()])

In [ ]:
ab_tmp = np.array([_abundance[1].as_matrix(), _abundance[2].as_matrix()])

In [ ]:
bad.shape

In [ ]:
tp_tmp.shape

In [ ]:
times = np.array([[3, 14, 21, 28], [3,14,21,28]])
#times = np.transpose(times[...,np.newaxis], axes=[0,2,1])

In [ ]:
times.shape

In [ ]:
repmat(times, 100, 1).reshape((2,100,4))

In [ ]:
times.shape

In [ ]:
tp_tmp.shape

# fit_ac_fc function

In [ ]:
def create_mask(counts, abundance, n_good=2): 
    pass_mask = pd.DataFrame(counts.values > abundance.values, columns=counts.columns)
    mask = pass_mask.groupby(level=0, axis=1).apply(lambda x: x.sum(axis=1) >= n_good)
    
    return mask 

def masked_cov(x,y): 
    term1 = np.ma.mean(np.ma.multiply(x,y), axis=1)
    term2 = np.multiply(np.ma.mean(x, axis=1), np.ma.mean(y, axis=1))
    
    return term1 - term2

In [ ]:
def fit_ac_fc_np(counts, abundance, times, n_good=2):
    #TODO: parse times
    
    mx_s, mt_s, v_s, f_s = [], [], [], []
    for count, ab in zip(counts, abundance): 
        mask = create_mask(count, ab, n_good=n_good)
        
        count_ma = np.ma.masked_array(count, mask=~mask)
        time_ma  = np.ma.masked_array(repmat(times, count.shape[0], 1), mask=~mask)
        
        mx = count_ma.mean(axis=1)
        mt = time_ma.mean(axis=1)
        v  = time_ma.var(axis=1)
        f = masked_cov(count_ma, time_ma)
        
        mx_s.append(mx)
        mt_s.append(mt)
        v_s.append(v_s)
        f_s.append(f)
        
    f_sum = np.zeros(counts[0].shape[0],)
    v_sum = np.zeros(counts[0].shape[0],)
    for f, v in zip(f_s, v_s): 
        f_sum = np.add(f_sum, f)
        v_sum = np.add(v_sum, v)
        
    fc = np.divide(f_sum, v_sum)
    
    ac_s = []
    for mx, mt in zip(mx, mt): 
        ac = mx - np.multiply(fc, mt)
        ac_s.append(ac)

In [ ]:
%time fit_ac_fc_np([_tps[1], _tps[2]], [_abundance[1], _abundance[2]], np.array([3, 14, 21, 28]))

In [ ]:
a = np.arange(0,10)

In [ ]:
np.power(2,a)

In [ ]:
np.__version__

In [ ]:
_data = _tps[1].values
_ab = _abundance[1].values

In [ ]:
_data > _ab

In [ ]:
data_masked = np.ma.masked_array(_data, mask=_data < _ab)
times_masked = np.ma.masked_array(repmat(times, data.shape[0], 1), mask=data_masked.mask)

In [ ]:
mx = data_masked.mean(axis=1)
mt = times_masked.mean(axis=1)
v  = times_masked.var(axis=1)
f = masked_cov(data_masked, times_masked)

In [ ]:
def fit_ac_fc(counts, abundance, n_good=2): 
    #Consider doing this in pure numpy 
    
    assert (counts.columns == abundance.columns).all()
    
    return create_mask(counts, abundance, n_good=n_good)

    # aggregate mask for "allbad"
    
    
    #return pass_mask

# Debug

In [36]:
from config import config
from fit_ac_fc import *
from fit_ac_fc import _cov

ab = load_abundance_thresholds(os.path.join(config.A549_test, \
        "A549_abundance_thresholds.txt"))
tps = load_timepoint_counts(os.path.join(config.A549_test, \
        "A549_timepoint_counts.txt"))

_tps, names = convert_timepoint_counts(tps)
_abundance = convert_abundance_thresholds(ab)

#For Testing only
#_tps = _tps.iloc[:100,:]
#_abundance = _abundance.iloc[:100,:]


times = np.array([[3, 14, 21, 28], [3,14,21,28]])
#times = np.transpose(times[...,np.newaxis], axes=[0,2,1])
times = repmat(times, _tps.shape[0], 1).reshape((times.shape[0],_tps.shape[0],times.shape[1]))

# fit_ac_fc_np(np.array([_tps[1].as_matrix(), _tps[2].as_matrix()]),
#              np.array([_abundance[1].as_matrix(), _abundance[2].as_matrix()]),
#              times)

counts = np.array([_tps[1].as_matrix(), _tps[2].as_matrix()])
ab = np.array([_abundance[1].as_matrix(), _abundance[2].as_matrix()])
times = times
n_good=2

In [40]:
bad = (counts > ab).sum(axis=2) < n_good
allbad = bad.all(axis=0)

counts_masked = np.ma.array(data=counts, mask=~(counts > ab))
time_masked = np.ma.array(data=times, mask=~(counts > ab))

mean_counts = np.ma.mean(counts_masked, axis=2).data
mean_time = np.ma.mean(time_masked, axis=2).data
var_time = np.ma.var(time_masked,axis=2).data
f = _cov(counts_masked, time_masked, axis=2).data

#fc = np.divide(f.sum(axis=0), var_time.sum(axis=0))
#fc[allbad] = 0

#ac = mean_counts - (fc*mean_time)
#ac[bad] = counts[bad,0]

In [44]:
bad.shape

(2, 24908)

In [34]:
ac[bad]

array([ 154.18218623])

In [45]:
bad = (counts > ab).sum(axis=2) < n_good
allbad = bad.all(axis=0)

counts_masked = np.ma.array(data=counts, mask=~(counts > ab))
time_masked = np.ma.array(data=times, mask=~(counts > ab))

mean_counts = np.ma.mean(counts_masked, axis=2).data
mean_time = np.ma.mean(time_masked, axis=2).data
var_time = np.ma.var(time_masked,axis=2).data
f = _cov(counts_masked, time_masked, axis=2).data

fc = np.divide(f.sum(axis=0), var_time.sum(axis=0))
fc[allbad] = 0

ac = mean_counts - (fc*mean_time)
ac[bad] = counts[bad,0]

alpha = -np.log2(np.power(2,ac).sum(axis=1))[...,np.newaxis]
ac = ac + alpha

lmbda = -np.log2(np.power(2, ac[...,np.newaxis] + fc[np.newaxis,...,np.newaxis]*times).sum(axis=1))
xfit = ac[...,np.newaxis] + fc[np.newaxis,...,np.newaxis]*times + lmbda[...,np.newaxis].transpose(0,2,1)

df = (counts > ab).sum(axis=2).sum(axis=0) - 2
numerator = np.sqrt(np.power(xfit - counts_masked, 2).sum(axis=2).sum(axis=0))
denominator = np.sqrt(np.power(time_masked - time_masked.mean(axis=2)[...,np.newaxis], 2).sum(axis=2).sum(axis=0))

sdfc = np.divide(numerator,denominator)

df_masked = np.ma.array(data=df, mask=~(df > 0))

tstat_masked = np.ma.divide(fc, np.ma.divide(sdfc, np.ma.sqrt(df_masked)))
tstat_masked.data[tstat_masked.mask] = 1

tstat = tstat_masked.data
p_t = np.array([2*t.cdf(i,j) if j > 0 else 1 for i,j in zip(tstat, df)])

/Users/Work/bin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
/Users/Work/bin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in power
/Users/Work/bin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log2
/Users/Work/bin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in add
/Users/Work/bin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


In [46]:
p_t

array([ 1.57735027,  1.64408232,  1.64408232, ...,  1.63678253,
        1.64408232,  1.        ])